# Pricing Options Using COS / FFT Methods of BS, Merton, Kou, and Heston

This workbook demonstrates the highly efficient pricing of European options using COS and FFT methods applied to the Black-Scholes (BS), Merton, Kou, and Heston models. It provides step-by-step guidance on how to calibrate these models, compute option prices across various strikes, and compare them to market-implied volatilities. The process is optimized for handling 0 DTE (zero days to expiry) options using high-frequency data.

## Loading 0DTE Option Data

The data loading process is handled by the `load_0dte_data` method, as demonstrated in the provided Python code. This method performs the following steps:

- Reads raw option data from CSV files using `pandas`.
- Processes timestamps and symbols to extract relevant information.

Although no explicit data preparation is required, this method sets up the foundation for accessing and analyzing the required market data for further processing.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import scipy.stats as st
import scipy.optimize as optimize

i = 1j    # imag unit
SEC_PER_YEAR = 365 * 24 * 3600 # seconds to years

def load_0dte_data(hour='08'):
    """
    Load 0DTE option data for BTC and ETH from CSV files.
    The data is filtered to include only options that are expiring today.
    Returns a tuple of (btc_df, eth_df).
    """
    btc_df = pd.read_csv(os.path.join(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/{hour}', f'btc_{hour}_0dte_data.csv'))
    eth_df = pd.read_csv(os.path.join(f'/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/Data/calibration_data/{hour}', f'eth_{hour}_0dte_data.csv'))

    # Convert the timestamps to UTC 
    btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'], utc=True)
    eth_df['timestamp'] = pd.to_datetime(eth_df['timestamp'], utc=True)
    return btc_df, eth_df


## Black-Scholes Functions:
The code includes functions to calculate Black-Scholes option prices for both call and put options. Additionally, the `implied_volatility` function is used to compute the volatility that corresponds to an observed market option price. It does this using a numerical solver to match the market price to the Black-Scholes formula. This step is critical for model calibration, as it enables the comparison of implied volatilities generated by the model to those observed in the market.


In [2]:
def extract_inputs_from_df(df):
    return (
        df['opt_type'].values,
        df['spot'].values,
        df['strike'].values,        
        df['time_to_maturity'].values,
        df['mark_price'].values,
        df['mark_iv'].values / 100.0  
        )

# –––––––––––––––– Black-Scholes Prices ––––––––––––––––
def bs_price(CP, S0, K, sigma, tau_sec, r):
    """Vectorised BS call/put price. tau_sec is time‑to‑expiry in seconds."""
    tau = np.asarray(tau_sec, dtype=float) / SEC_PER_YEAR          # years
    CP  = np.asarray(CP, dtype=str)

    d1  = (np.log(S0 / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2  = d1 - sigma * np.sqrt(tau)

    price = np.where(
        CP == 'call',
        st.norm.cdf(d1) * S0 - st.norm.cdf(d2) * K * np.exp(-r * tau),
        st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1) * S0
    )
    return price

# –––––––––––––––– Black-Scholes Greeks & IV ––––––––––––––––
def bs_delta(CP, S0, K, sigma, tau_sec, r):
    tau = tau_sec / SEC_PER_YEAR
    d1  = (np.log(S0 / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return np.where(CP == 'call', st.norm.cdf(d1), st.norm.cdf(d1) - 1)

def bs_gamma(S0, K, sigma, tau_sec, r):
    tau = tau_sec / SEC_PER_YEAR
    d1  = (np.log(S0 / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return st.norm.pdf(d1) / (S0 * sigma * np.sqrt(tau))

def bs_vega(S0, K, sigma, tau_sec, r):
    tau = tau_sec / SEC_PER_YEAR
    d1  = (np.log(S0 / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return S0 * st.norm.pdf(d1) * np.sqrt(tau)

def bs_impliedvol(CP, marketPrice, K, tau, S0, r):
    func = lambda sigma: np.power(bs_price(CP, S0, K, sigma, tau, r) - marketPrice, 1.0)
    impliedVol = optimize.newton(func, 0.7, tol=1e-9)
    #impliedVol = optimize.brent(func, brack= (0.05, 2))
    return impliedVol

## Characteristic Functions Used for Pricing

The Python wrappers

```python
chf_merton(u, ..., S0, K)
chf_kou(u, ..., S0, K)
chf_heston(u, ..., S0, K)
```

each return the characteristic function of **log-moneyness**, defined as

$$
X_\tau = \log\left(\frac{S_\tau}{K}\right), \qquad
x = \log\left(\frac{S_0}{K}\right), \qquad
\tau = \frac{\texttt{tau\_sec}}{365 \times 24 \times 3600}.
$$

All wrappers return:

$$
\phi_X(u, \tau) = \mathbb{E}\left[e^{iu X_\tau} \right] = e^{iux} \cdot \varphi_{\text{model}}(u, \tau)
$$

The phase shift $e^{iux}$ is implemented as:

```python
shift = np.exp(i * u * np.log(S0 / K))
```

---

### Merton Jump-Diffusion Model

```python
phi_return = phi_merton(...)
phi_X      = shift * phi_return
```

$$
\phi_X^{\text{Merton}}(u, \tau) = e^{iux} \cdot \exp\left[
    iu\mu\tau - \frac{1}{2}\sigma^2 u^2 \tau + \xi\tau \left(e^{iu\mu_J - \frac{1}{2}\sigma_J^2 u^2} - 1\right)
\right]
$$

with

$$
\mu = r - \frac{1}{2}\sigma^2 - \underbrace{\xi\left(e^{\mu_J + \frac{1}{2}\sigma_J^2} - 1\right)}_{\bar{\omega}}.
$$

**Interpretation**  
Black-Scholes dynamics plus Poisson jumps of normal size $\mathcal{N}(\mu_J, \sigma_J^2)$, arriving at rate $\xi$.

---

### Kou Double-Exponential Jump-Diffusion Model

```python
phi_return = phi_kou(...)
phi_X      = shift * phi_return
```

$$
\phi_X^{\text{Kou}}(u, \tau) = e^{iux} \cdot \exp\left[
    iu\mu\tau - \frac{1}{2}\sigma^2 u^2 \tau + \xi\tau \left( \frac{p_1\alpha_1}{\alpha_1 - iu} + \frac{p_2\alpha_2}{\alpha_2 + iu} - 1 \right)
\right]
$$

with

$$
p_2 = 1 - p_1, \qquad
\mu = r - \frac{1}{2}\sigma^2 - \xi \left( \frac{p_1\alpha_1}{\alpha_1 - 1} + \frac{p_2\alpha_2}{\alpha_2 + 1} - 1 \right)
$$

**Interpretation**  
Jump sizes follow a double-exponential distribution, allowing for asymmetric up/down moves with heavier tails.

---

### Heston Stochastic Volatility Model

```python
phi_return = phi_heston(...)
phi_X      = shift * phi_return
```

$$
\phi_X^{\text{Heston}}(u, \tau; v_0) = e^{iux} \cdot \exp\left[ A(u, \tau) + C(u, \tau) v_0 \right]
$$

with:

\begin{align}
D_1 &= \sqrt{(\kappa - i\rho\gamma u)^2 + (u^2 + iu)\gamma^2} \\
g   &= \frac{\kappa - i\rho\gamma u - D_1}{\kappa - i\rho\gamma u + D_1} \\
C(u, \tau) &= \frac{1 - e^{-D_1 \tau}}{\gamma^2(1 - g e^{-D_1 \tau})}(\kappa - i\rho\gamma u - D_1) \\
A(u, \tau) &= iu r \tau + \frac{\kappa \bar{v}}{\gamma^2} \left[
    (\kappa - i\rho\gamma u - D_1)\tau - 2 \log\left( \frac{1 - g e^{-D_1 \tau}}{1 - g} \right)
\right]
\end{align}

**Interpretation**  
Variance $v_t$ follows a mean-reverting CIR process. Parameters ($\kappa, \bar{v}, \gamma, \rho$) control long-run variance, vol-of-vol, and spot-vol correlation.

---

### Practical Note

Each wrapper already includes the phase term $e^{iux}$.  
So the returned $\phi_X(u, \tau)$ can be passed **directly** into a COS or FFT pricer, without modification.


In [3]:
# ------------------  MERTON  ------------------
def chf_merton(u, sigma, xi, muJ, sigmaJ, r, tau_sec):
    tau = tau_sec / SEC_PER_YEAR
    omega_bar = xi * (np.exp(muJ + 0.5*sigmaJ**2) - 1)
    mu = r - 0.5*sigma**2 - omega_bar
    return np.exp(i*u*mu*tau -0.5*sigma**2*u**2*tau +
                  xi*tau*(np.exp(i*muJ*u - 0.5*sigmaJ**2*u**2) - 1))

def phi_merton(sigma, xi, muJ, sigmaJ, r, tau_sec, S0, K):
    return lambda u: np.exp(i*u*(np.log(S0) - np.log(K)))  * chf_merton(u, sigma, xi, muJ, sigmaJ, r, tau_sec)

# ------------------  KOU  ---------------------
def chf_kou(u, sigma, xi, p1, alpha1, alpha2, r, tau_sec):
    p2 = 1 - p1
    tau = tau_sec / SEC_PER_YEAR
    omega_bar = xi * (p1*alpha1/(alpha1-1) + (1-p1)*alpha2/(alpha2+1) - 1)
    mu = r - 0.5*sigma**2 - omega_bar
    return np.exp(i*u*mu*tau-0.5*sigma**2*u**2*tau +
                  xi*tau*((p1*alpha1)/(alpha1 - i*u) +
                          (p2*alpha2)/(alpha2 + i*u) - 1))

def phi_kou(sigma, xi, p1, alpha1, alpha2, r, tau_sec, S0, K):
    return lambda u: np.exp(i*u*(np.log(S0) - np.log(K))) * chf_kou(u, sigma, xi, p1, alpha1, alpha2, r, tau_sec)

# ------------------  HESTON  ------------------
def chf_heston(u, tau_sec, r, kappa, v_bar, gamma, rho, v0):
    tau = tau_sec / SEC_PER_YEAR 
    d1 = np.sqrt((kappa - i*rho*gamma*u)**2 + (u**2 + i*u)*gamma**2)
    g  = (kappa - i*rho*gamma*u - d1) / (kappa - i*rho*gamma*u + d1)
    term_r  = i*u*r*tau
    term_v0 = (v0 / gamma**2) * ((1 - np.exp(-d1*tau)) /
                                 (1 - g*np.exp(-d1*tau))) * (kappa - i*rho*gamma*u - d1)
    term_bar= (kappa*v_bar / gamma**2) * (tau*(kappa - i*rho*gamma*u - d1) -
                                          2*np.log((1 - g*np.exp(-d1*tau))/(1 - g)))
    return np.exp(term_r + term_v0 + term_bar)

def phi_heston(kappa, v_bar, gamma, rho, v0, r, tau_sec, S0, K):
    return lambda u: np.exp(i*u*(np.log(S0) - np.log(K))) * chf_heston(u, tau_sec, r, kappa, v_bar, gamma, rho, v0)

## COS Valuation Method (`cos_valuation`):
The `cos_valuation` method is the core pricing engine in the code. It uses the COS method, a numerical technique that approximates the option price by expanding the characteristic function into a cosine series. This method is efficient and accurate, especially for pricing European options across a range of strikes. The method takes the characteristic function from one of the models, the spot price, strike price, risk-free rate, time to maturity, and the option type (call or put) as inputs. The integration range `[a, b]` is determined based on the cumulants of the log-return distribution, ensuring accuracy in pricing.


In [ ]:
# ------------------  Cumulants  ------------------
def merton_cumulants(tau, r, sigma, xi, muJ, sigmaJ):
    """ Calculate the cumulants for the Merton model. """
    omega_bar = xi * (np.exp(muJ + 0.5 * sigmaJ**2) - 1)
    c1 = tau * (r - omega_bar - 0.5 * sigma**2 + xi * muJ)
    c2 = tau * (sigma**2 + xi * muJ**2 + sigmaJ**2 * xi)
    c4 = tau * xi * (muJ**4 + 6 * muJ**2 * sigmaJ**2 + 3 * sigmaJ**4 * xi)
    # c4 = tau * xi * (muJ**4 + 6 * muJ**2 * sigmaJ**2 + 3 * sigmaJ**4)
    return c1, c2, c4

def kou_cumulants(tau, r, sigma, xi, alpha1, alpha2, p1):
    """ Calculate the cumulants for the Heston model. """
    p2 = 1 - p1
    omega_bar = xi * ((p1*alpha1)/(alpha1-1) + (p2*alpha2)/(alpha2+1) - 1)
    c1 = tau * (r - omega_bar - 0.5 * sigma**2 + ((xi * p1)/alpha1 - (xi * p2)/alpha2))
    c2 = tau * (sigma**2 + 2 * (xi * p1)/ alpha1**2 + 2 * (xi * p2)/alpha2**2)
    c4 = 24 * tau * xi * (p1 / alpha1**4 + p2 / alpha2**4) 
    return c1, c2, c4

# ------------------  COS TRUNCATION WINDOW  ------------------
def truncation_window(S0, K, tau, model, r, theta, L=8):
    """ Compute COS truncation window [a, b] for different models. """
    log_ratio = np.log(S0 / K)

    if model.lower() == 'merton':
        sigma, xi, muJ, sigmaJ = theta
        c1, c2, c4 = merton_cumulants(tau, r, sigma, xi, muJ, sigmaJ)
        a = log_ratio + c1 - L*np.sqrt(c2 + np.sqrt(c4))
        b = log_ratio + c1 + L*np.sqrt(c2 + np.sqrt(c4))

    elif model.lower() == 'kou':
        sigma, xi, p1, a1, a2 = theta
        c1, c2, c4 = kou_cumulants(tau, r, sigma, xi, a1, a2, p1)
        a = log_ratio + c1 - L*np.sqrt(c2 + np.sqrt(c4))
        b = log_ratio + c1 + L*np.sqrt(c2 + np.sqrt(c4))

    elif model.lower() == 'heston':
        a = log_ratio - L*np.sqrt(tau)
        b = log_ratio + L*np.sqrt(tau)

    else:
        raise ValueError("unknown model")

    return a, b

# ------------------  COS PAYOFF COEFFICIENTS  ------------------
def chi_psi(k, a, b, c, d):
    """Compute the Xi(c,d) and Psi(c,d) vectors used in the COS method."""
    k = np.asarray(k, dtype=float)
    omega = k * np.pi / (b - a)

    # Allocate output arrays
    chi = np.zeros_like(k, dtype=float)
    psi = np.zeros_like(k, dtype=float)

    # k = 0 terms 
    chi[0] = np.exp(d) - np.exp(c)
    psi[0] = d - c

    # k >= 1 terms
    if k.size > 1:               
        k_nz      = k[1:]
        omega_nz  = omega[1:]
        denom_nz  = 1.0 + omega_nz**2

        chi[1:] = (
            np.cos(omega_nz * (d - a)) * np.exp(d)
          + omega_nz * np.sin(omega_nz * (d - a)) * np.exp(d)
          - np.cos(omega_nz * (c - a)) * np.exp(c)
          - omega_nz * np.sin(omega_nz * (c - a)) * np.exp(c)
        ) / denom_nz

        psi[1:] = (b - a) / (k_nz * np.pi) * (
            np.sin(omega_nz * (d - a)) - np.sin(omega_nz * (c - a))
        )
    return chi, psi

def payoff_coefficients(option_type, K, k, a, b):
    """Return the array H_k for a European call or put."""
    if option_type.lower() == 'call':
        c, d = 0.0, b
        chi, psi = chi_psi(k, a, b, c, d)
        H_k = (2.0 / (b - a)) * K * (chi - psi)
    else:                                       # put
        c, d = a, 0.0
        chi, psi = chi_psi(k, a, b, c, d)
        H_k = (2.0 / (b - a)) * K * (-chi + psi)

    return H_k      


# ───────── COS PRICER ───────────────────────────────────────────
def cos_price_single(model, option_type, S0, K, tau_sec, r, theta, N=256, L=8):
    """COS price for one European call or put."""
    tau = tau_sec / SEC_PER_YEAR
    k   = np.arange(N, dtype=float)           # 0 … N‑1

    # Model‑specific CF & [a,b]
    if model == 'merton':
        sigma, xi, muJ, sigmaJ = theta
        a, b = truncation_window(S0, K, tau, 'merton', r, theta, L=L)
        phi_func  = phi_merton(sigma, xi, muJ, sigmaJ, r, tau_sec, S0, K)

    elif model == 'kou':
        sigma, xi, p1, a1, a2 = theta 
        a, b = truncation_window(S0, K, tau, 'kou', r, theta, L=L)
        phi_func = phi_kou(sigma, xi, p1, a1, a2, r, tau_sec, S0, K)

    elif model == 'heston':
        kappa, vbar, gamma, rho, v0 = theta
        a, b = truncation_window(S0, K, tau, 'heston', r, theta, L=L)
        phi_func = phi_heston(kappa, vbar, gamma, rho, v0, r, tau_sec, S0, K)
    else:
        raise ValueError("model must be 'merton', 'kou', or 'heston'")

    # --- Chi, Psi payoff coefficients --------------------------------
    H_k = payoff_coefficients(option_type, K, k, a, b)

    u       = k * np.pi / (b - a)
    phi_u   = phi_func(u)                     # CF on grid
    weights = np.real(phi_u * np.exp(-1j * u * a))
    weights[0] *= 0.5                        # 0.5 weight for k=0

    price = np.exp(-r * tau) * np.sum(weights * H_k)
    return price

def cos_price_batch(model, S0, K_vec, tau_sec_vec, r, option_type_vec, theta,
                    N=256, L=8):
    """Price many options (mixed calls/puts) via a simple loop."""
    prices = np.empty_like(K_vec, dtype=float)
    for j, (K, tau_s, opt) in enumerate(zip(K_vec, tau_sec_vec, option_type_vec)):
        prices[j] = cos_price_single(model, opt, S0, K, tau_s, r, theta, N=N, L=L)
    return prices


## Parameter Calibration Methods (`calibrate_merton`, `calibrate_kou`, `calibrate_heston`):
These methods are responsible for calibrating the model parameters to match market data. The calibration process involves two steps:
- **Global Search:** A global search is performed using `scipy.optimize.differential_evolution`, which explores the parameter space to find a promising region where the optimal parameters are likely to be.
- **Local Refinement:** Once a promising region is found, local refinement is done using `scipy.optimize.minimize` with the BFGS algorithm, which fine-tunes the parameters to minimize the RMSE.

These methods iteratively adjust the model parameters, ensuring that the model’s implied volatilities closely match the observed market-implied volatilities. Predefined bounds for each model parameter ensure that the optimizer searches within realistic and valid ranges.


In [ ]:
# Initial model configuration for calibration (initial guess, lower bound and upper bound)
MODEL_CFG = {
    'merton': (
        np.array([0.40, 0.10, -0.10, 0.25]),      # sigma, xi, muJ, sigmaJ
        np.array([0.05, 0.00, -1.50, 0.05]),
        np.array([3.00, 1.00,  0.00, 1.00])
    ),
    'kou': (
        np.array([0.35, 1.00, 5.00, 0.20, 0.50]), # sigma, xi, a1, a2, p1
        np.array([0.05, 0.00, 1.10, 0.10, 0.00]),
        np.array([3.00, 5.00, 9.00, 5.00, 1.00])
    ),
    'heston': (
        np.array([1.50, 0.30, 0.50, -0.60, 0.05]), # kappa, v_bar, gamma, rho, v0
        np.array([0.10, 0.01, 0.05, -0.99, 0.01]),
        np.array([5.00, 1.00, 2.00,  0.99, 1.00])
    )
}

def iv_newton(price, CP, S0, K, tau_sec, r, sigma_init=0.3, tol=1e-8, max_iter=10):
    """Vectorised Newton–Raphson implied‑vol solver."""
    sigma = np.full_like(price, sigma_init, dtype=float)

    for _ in range(max_iter):
        diff  = price - bs_price(CP, S0, K, sigma, tau_sec, r)
        vega  = bs_vega(S0, K, sigma, tau_sec, r)
        sigma += diff / np.where(vega > 1e-12, vega, np.inf)

        if np.all(np.abs(diff) < tol):
            break

    sigma = np.where((sigma > 0) & (sigma < 5), sigma, np.nan)
    return sigma

def rmse_iv(theta, model, S0, K, tau_sec, CP, iv_mkt, r=0.0):
    """RMSE between model IVs and market IVs for the specified model."""
    price_mod = cos_price_batch(model, S0, K, tau_sec, r, CP, theta)
    iv_mod    = iv_newton(price_mod, CP, S0, K, tau_sec, r, sigma_init=iv_mkt)

    m = np.isfinite(iv_mod) & np.isfinite(iv_mkt)
    return 100 * np.sqrt(np.mean((iv_mod[m] - iv_mkt[m])**2))


def calibrate_snapshot(df_snap, model, r=0.0):
    """Calibrate model parameters for one day's option surface."""
    if model not in MODEL_CFG:
        raise ValueError(f"unknown model: {model}")

    theta_0, lowerbound, upperbound = MODEL_CFG[model]
    CP, S0_vec, K, tau, mkt_price, iv_mkt = extract_inputs_from_df(df_snap)
    S0 = float(S0_vec[0])

    loss = lambda theta: rmse_iv(theta, model, S0, K, tau, CP, iv_mkt, r)
    res  = optimize.minimize(loss, theta_0, bounds=optimize.Bounds(lowerbound, upperbound), method='L-BFGS-B')

    return {
        'model':  model,
        'theta':  res.x,
        'iv_rmse': res.fun,
        'n_strk':  len(df_snap),
        'success': res.success,
        'message': res.message
    }

if __name__ == '__main__':
    df_btc, _ = load_0dte_data('08')
    df_btc    = df_btc.dropna(subset=['mark_price', 'mark_iv'])

    fits = []
    for date, snap in df_btc.groupby(df_btc['timestamp'].dt.date):
        for pricing_model in ['merton', 'kou', 'heston']:           
            fit = calibrate_snapshot(snap, pricing_model)
            fit['date'] = date
            fits.append(fit)

            theta = ', '.join(f'{x:.3f}' for x in fit['theta'])
            print(f"{date} [{pricing_model}]  RMSE={fit['iv_rmse']:.2f}  theta=[{theta}]")

    # pd.DataFrame(fits).to_csv('daily_fits.csv', index=False)



2021-09-07 [merton]  RMSE=11.84  theta=[0.806, 1.000, -0.177, 0.050]


/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_25284/831809399.py:16: RuntimeWarning: divide by zero encountered in scalar divide
  omega_bar = xi * (p1*alpha1/(alpha1-1) + (1-p1)*alpha2/(alpha2+1) - 1)
/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_25284/831809399.py:18: RuntimeWarning: invalid value encountered in multiply
  return np.exp(i*u*mu*tau-0.5*sigma**2*u**2*tau +
/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3859: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/joris/Documents/Master QF/Thesis/optimal-gamma-hedging/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/j1/sv4t8szd51jgm826w2bv1vyr0000gn/T/ipykernel_25284/831809399.py:19: RuntimeWarning: invalid value encountered in divide
  xi*tau*((p1*alpha1)/(alp

2021-09-07 [kou]  RMSE=nan  theta=[0.350, 1.000, 5.000, 0.200, 0.500]
2021-09-07 [heston]  RMSE=16.82  theta=[4.320, 0.660, 0.151, -0.928, 0.806]
2021-09-08 [merton]  RMSE=12.57  theta=[1.097, 0.201, -0.535, 0.050]
2021-09-08 [kou]  RMSE=nan  theta=[0.350, 1.000, 5.000, 0.200, 0.500]
2021-09-08 [heston]  RMSE=26.28  theta=[0.100, 0.010, 0.050, -0.990, 1.000]
2021-09-09 [merton]  RMSE=11.51  theta=[0.782, 0.002, -0.041, 0.201]
2021-09-09 [kou]  RMSE=nan  theta=[0.350, 1.000, 5.000, 0.200, 0.500]
2021-09-09 [heston]  RMSE=37.04  theta=[1.500, 0.300, 0.500, -0.600, 0.050]
2021-09-10 [merton]  RMSE=12.02  theta=[0.785, 1.000, -0.234, 0.050]
2021-09-10 [kou]  RMSE=nan  theta=[0.350, 1.000, 5.000, 0.200, 0.500]
2021-09-10 [heston]  RMSE=22.38  theta=[1.539, 0.369, 2.000, -0.990, 0.884]
2021-09-11 [merton]  RMSE=9.38  theta=[0.764, 1.000, -0.167, 0.050]
2021-09-11 [kou]  RMSE=nan  theta=[0.350, 1.000, 5.000, 0.200, 0.500]
2021-09-11 [heston]  RMSE=17.60  theta=[4.326, 1.000, 0.259, -0.780, 0.

KeyboardInterrupt: 

## RMSE Loss Function (`calculate_rmse_iv`):
The `calculate_rmse_iv` function is crucial for calibrating the model parameters. It computes the Root Mean Squared Error (RMSE) between the implied volatilities generated by the model and the actual market-implied volatilities. The function calculates theoretical prices using the `cos_valuation` method, converts these prices into model-implied volatilities, and compares them to the market values. The objective is to minimize this RMSE during the calibration process, ensuring the model parameters provide the best fit to the market data.
